### Imports

In [1]:
import pandas as pd
import datetime
import os
import sys
import math
import matplotlib.pyplot as plt
import numpy as np
import warnings
from pandas.errors import SettingWithCopyWarning
from pandas import to_datetime
import statsmodels.api as sm
from statsmodels import regression
from sklearn.linear_model import LinearRegression


warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

### Parameters

In [2]:
buy_threshold = 0.1
sell_threshold = 0.1
starting_balance = 1000000
price_cap_lower = 2
investment_ratio = 0.00005
long_short_ratio = 0.05

run_optimizing = False
google_colab = False

### Data loading

In [3]:
if google_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    !pip install -U -q PyDrive
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials
    # Authenticate and create the PyDrive client.
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    id = "1PZQLyOiOpKtLt5BVAvWwQEnxZJzvLcD7"
    downloaded = drive.CreateFile({'id':id}) 
    downloaded.GetContentFile('Full trading data - All models.csv')  
    df = pd.read_csv('Full trading data - All models.csv')
else:
    # Set the path to the root directory
    path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
    # Read dataframes using Dask
    df = pd.read_csv(path + '/data/trading/Full trading data - All models.csv')

In [4]:
# Drop Expire_date column
df.drop('Expire_date', axis=1, inplace=True)

In [5]:
# Filter so its just data for 2015
#df = df[(df["Quote_date"] >= "2015-01-01") & (df["Quote_date"] <= "2018-12-31")]
#df = df[(df["Expiry_date"] >= "2015-01-01") & (df["Expiry_date"] <= "2018-12-31")]

# Choos 5% of the data randomly
df = df.sample(frac=0.05, random_state=1)

In [6]:
lstm_mlp = True
bs_rolling = False
bs_garch = False
bs_iv = False
heston = False

if lstm_mlp:
    # Out of Quote_date	Expire_date	Price	Bid	Ask	Underlying_last	Strike	TTM	Delta	IV	R	Price_drop	Rolling	GARCH	BS-IV	Heston	LSTM-MLP, drop Rolling, GARCH, BS-IV, Heston
    df = df[['Quote_date', 'Expiry_date', 'Price', 'Bid', 'Ask', 'Underlying_last', 'Strike', 'TTM', 'Delta', 'IV', 'R', 'Price_drop', 'Option_ID', 'LSTM-MLP']]
    # Rename LSTM-MLP to Prediction
    df = df.rename(columns={'LSTM-MLP': 'Prediction'})
elif bs_rolling:
    df = df[['Quote_date', 'Expiry_date', 'Price', 'Bid', 'Ask', 'Underlying_last', 'Strike', 'TTM', 'Delta', 'IV', 'R', 'Price_drop', 'Option_ID', 'Rolling']]
    df = df.rename(columns={'Rolling': 'Prediction'})
elif bs_garch:
    df = df[['Quote_date', 'Expiry_date', 'Price', 'Bid', 'Ask', 'Underlying_last', 'Strike', 'TTM', 'Delta', 'IV', 'R', 'Price_drop', 'Option_ID', 'GARCH']]
    df = df.rename(columns={'GARCH': 'Prediction'})
elif bs_iv:
    df = df[['Quote_date', 'Expiry_date', 'Price', 'Bid', 'Ask', 'Underlying_last', 'Strike', 'TTM', 'Delta', 'IV', 'R', 'Price_drop', 'Option_ID', 'BS-IV']]
    df = df.rename(columns={'BS-IV': 'Prediction'})
elif heston:
    df = df[['Quote_date', 'Expiry_date', 'Price', 'Bid', 'Ask', 'Underlying_last', 'Strike', 'TTM', 'Delta', 'IV', 'R', 'Price_drop', 'Option_ID', 'Heston']]
    df = df.rename(columns={'Heston': 'Prediction'})

### Data processing

In [7]:
df["TTM"] = (df["TTM"]*365).astype(int)

In [8]:
print(len(df))

437969


# Trading functions

In [9]:
def generate_buy_sell_signals(df, buy_threshold, sell_threshold):
    buy_signal = (df['Prediction'] - df['Ask']) / df['Ask'] >= buy_threshold
    sell_signal = (df['Bid'] - df['Prediction']) / df['Bid'] >= sell_threshold
    return buy_signal, sell_signal


def trader(df, buy_signal, sell_signal, starting_balance, price_cap_lower, investment_ratio = 0.002, long_short_ratio= 0.02, fee = 0.001):
    df['Expiry_date'] = pd.to_datetime(df['Expiry_date'])
    df['Quote_date'] = pd.to_datetime(df['Quote_date'])
    df = df[df['Expiry_date'] <= df['Quote_date'].max()]
    
    df['Signal'] = 0
    df.loc[buy_signal, 'Signal'] = 1
    df.loc[sell_signal, 'Signal'] = -1

    df['Position_this_opt'] = 0
    df['Balance'] = -1000000    # Makes it easy to spot if balance has not been updated correctly

    df = df.sort_values('Quote_date')
    df['Option_ID'] = df['Option_ID'].astype(str)

    position_dict = {id:0 for id in df['Option_ID'].unique()}
    first_date = df['Quote_date'].min()
    total_balance_dict = {first_date: starting_balance}

    number_of_long_positions = 0
    number_of_short_positions = 0

    def operation(row):
        nonlocal number_of_long_positions
        nonlocal number_of_short_positions
        if row['Quote_date'] not in total_balance_dict.keys():
            total_balance_dict[row['Quote_date']] = total_balance_dict[max(total_balance_dict.keys())]

        balance = total_balance_dict[row['Quote_date']]
        position = position_dict[row['Option_ID']]

        # If price below cap, do nothing
        if row["Price"] < price_cap_lower:
            row["Position_this_opt"] = position
            row["Balance"] = balance
            position_dict[row['Option_ID']] = position
            total_balance_dict[row['Quote_date']] = balance
            return row

        # Calculate net ratio
        if number_of_long_positions + number_of_short_positions == 0:
            net_ratio = 0
        else:
            net_ratio = (number_of_long_positions - number_of_short_positions) / (number_of_long_positions + number_of_short_positions)

        # Buy 
        if row['Signal'] == 1 and position <= 0 and row["Quote_date"] != row["Expiry_date"] and balance > 100 and net_ratio < long_short_ratio:
            amount_to_invest = balance * investment_ratio
            if amount_to_invest <= balance:
                balance -= amount_to_invest * (1 + fee)
                position += amount_to_invest / row['Ask']
                number_of_long_positions += 1

        # Sell
        elif row['Signal'] == -1 and position >= 0 and row["Quote_date"] != row["Expiry_date"] and balance > 100 and net_ratio > -long_short_ratio:
            amount_to_sell = balance * investment_ratio
            balance += amount_to_sell * (1 - fee)
            position -= amount_to_sell / row['Bid']
            number_of_short_positions += 1


        # Expire
        elif row["Quote_date"] == row["Expiry_date"] and position != 0:
            intrinsic_value = max(0, row['Underlying_last'] - row['Strike'])
            adjustment = intrinsic_value if position > 0 else -intrinsic_value
            balance += adjustment * abs(position)
            position = 0
            if position < 0:
                number_of_short_positions -= 1
            elif position > 0:
                number_of_long_positions -= 1
        
        
        row['Position_this_opt'] = position
        position_dict[row['Option_ID']] = position
        total_balance_dict[row['Quote_date']] = balance
        row['Balance'] = balance

        # Print progress
        if row.name % 100000 == 0:
            print(row["Quote_date"])
            print("Number of long positions: ", number_of_long_positions)
            print("Number of short positions: ", number_of_short_positions)
            print("Balnace: ", balance)
        return row

    df = df.apply(operation, axis=1)
    df.loc[(df['Position_this_opt'] == 0), 'Balance'] = df['Quote_date'].map(total_balance_dict)
    return df

def calculate_options_value(df):
    df['Options_value'] = 0
    for date in df['Quote_date'].unique():
        options_value_sum = (df.loc[df['Quote_date'] == date, 'Position_this_opt'] * df.loc[df['Quote_date'] == date, 'Price']).sum()
        df.loc[df['Quote_date'] == date, 'Options_value'] = options_value_sum
    df['Total_value'] = df['Balance'] + df['Options_value']
    return df

### Analysis functions

In [10]:
def sharpe_ratio(df):
    df_group = df.groupby('Quote_date').last()
    df_group['Excess_Returns'] = df_group['Total_value'].pct_change() - 0.02 / 252
    sharpe_ratio = np.sqrt(252) * (df_group["Excess_Returns"].mean() / df_group["Excess_Returns"].std())
    return sharpe_ratio

In [11]:
def sharpe_ratio_monthly(df):
    df_sharpe = df.copy()
    # Get YYYY-MM for each row
    df_sharpe["YYYY-MM"] = df_sharpe["Quote_date"].dt.strftime("%Y-%m")
    df_sharpe = df_sharpe.groupby("YYYY-MM").last()
    df_sharpe["Excess_Returns"] = df_sharpe["Total_value"].pct_change() - 0.02 / 12
    sharpe_ratio = np.sqrt(12) * (df_sharpe["Excess_Returns"].mean() / df_sharpe["Excess_Returns"].std())
    return sharpe_ratio 

In [12]:
def capm_alpha(df_capm):
    # Find alpha in the CAPM model using linear regression and grouping df by Quote_date, then taking the alpha of Total_value compared to Underlying_last which is the S&P500
    df_capm = df_capm.groupby('Quote_date').last()
    df_capm = df_capm[['Total_value', 'Underlying_last']]
    df_capm = df_capm.dropna()
    df_capm['Underlying_last'] = df_capm['Underlying_last'].pct_change()
    df_capm['Total_value'] = df_capm['Total_value'].pct_change()
    df_capm = df_capm.dropna()
    df_capm = df_capm.reset_index()
    df_capm = df_capm.drop(columns=['Quote_date'])
    df_capm = df_capm.dropna()
    X = df_capm['Underlying_last'].values.reshape(-1, 1)
    y = df_capm['Total_value'].values.reshape(-1, 1)
    reg = LinearRegression().fit(X, y)
    return reg.intercept_[0] * 252

In [13]:
def capm_beta(df_capm):
    # Find beta in the CAPM model using linear regression and grouping df by Quote_date, then taking the beta of Total_value compared to Underlying_last which is the S&P500
    df_capm = df_capm.groupby('Quote_date').last()
    df_capm = df_capm[['Total_value', 'Underlying_last']]
    df_capm = df_capm.dropna()
    df_capm['Underlying_last'] = df_capm['Underlying_last'].pct_change()
    df_capm['Total_value'] = df_capm['Total_value'].pct_change()
    df_capm = df_capm.dropna()
    df_capm = df_capm.reset_index()
    df_capm = df_capm.drop(columns=['Quote_date'])
    df_capm = df_capm.dropna()
    X = df_capm['Underlying_last'].values.reshape(-1, 1)
    y = df_capm['Total_value'].values.reshape(-1, 1)
    reg = LinearRegression().fit(X, y)
    return reg.coef_[0][0] * 252

In [14]:
def max_drawdown(df):
    df_group = df.groupby('Quote_date').last()
    df_group['Roll_Max'] = df_group['Total_value'].cummax()
    df_group['Drawdown'] = df_group['Total_value'] / df_group['Roll_Max'] - 1.0
    df_group['Drawdown'].min()
    return df_group['Drawdown'].min() * 100

In [15]:
def print_results(df, starting_balance):
    ending_balance = df['Balance'].iloc[-1]
    profit = ending_balance - starting_balance
    num_trades = df[df["Price"] > 1]["Signal"].abs().sum()
    profit_per_trade = profit / num_trades

    df['Quote_date'] = pd.to_datetime(df['Quote_date'])
    trading_days = df['Quote_date'].unique().shape[0] - 1
    profit_per_day = profit / trading_days
    num_days = (df['Quote_date'].max() - df['Quote_date'].min()).days

    annualized_return = ((ending_balance / starting_balance)**(365/num_days) - 1)*100

    print(f"Starting balance: {starting_balance}")
    print(f"Ending balance: {ending_balance.round(0)}")
    print(f"Profit: {profit.round(0)}")
    print(f"Profit per trade: {profit_per_trade.round(2)}")
    print(f"Profit per trading day: {profit_per_day.round(2)}")
    print(f"Sharpe ratio: {sharpe_ratio_monthly(df).round(2)}")
    print(f"CAPM alpha: {capm_alpha(df).round(2)}")
    print(f"Annualized return in percent: {annualized_return.round(2)}")
    print(f"Max drawdown: {max_drawdown(df).round(2)}")

In [16]:
def plot(df):
    # Just keep one row per Quote_date, and that should be the last row
    df = df.groupby('Quote_date').last()
    plt.plot(df['Total_value'], label='Total value')
    plt.plot(df['Balance'], label='Cash')
    plt.plot(df['Options_value'], label='Net value of options')
    plt.title('Portfolio value over time')
    # Make plot smaller
    ax = plt.gca()
    plt.xticks(rotation=45)
    plt.legend()
    plt.show()

## Run the code

In [23]:
# Initialize an empty DataFrame to store the results
results = pd.DataFrame(columns=["Year", "Starting balance", "Ending balance", "Profit", "Profit per trade", "Profit per trading day", 
                                "Sharpe ratio", "CAPM alpha", "Annualized return in percent", "Max drawdown"])

# Get the list of years present in the data
df['Quote_date'] = pd.to_datetime(df['Quote_date'])
years = df['Quote_date'].dt.year.unique()

# Sort the years
years.sort()

# Iterate over each year
for year in years:
    df_year = df[df['Quote_date'].dt.year == year].copy()
    
    if not df_year.empty:
        # Execute operations
        buy_signal, sell_signal = generate_buy_sell_signals(df_year, buy_threshold, sell_threshold)
        df_year = trader(df_year, buy_signal, sell_signal, starting_balance, price_cap_lower, investment_ratio, long_short_ratio)
        df_year = calculate_options_value(df_year)

        # Calculate metrics
        ending_balance = df_year['Balance'].iloc[-1]
        profit = ending_balance - starting_balance
        num_trades = df_year[df_year["Price"] > 1]["Signal"].abs().sum()
        profit_per_trade = profit / num_trades

        trading_days = df_year['Quote_date'].unique().shape[0] - 1
        profit_per_day = profit / trading_days
        num_days = (df_year['Quote_date'].max() - df_year['Quote_date'].min()).days

        annualized_return = ((ending_balance / starting_balance)**(365/num_days) - 1)*100

        plot(df_year)

        # Create a DataFrame for this year's metrics
        year_metrics = pd.DataFrame({
            "Year": [year], 
            "Starting balance": [starting_balance], 
            "Ending balance": [ending_balance.round(0)], 
            "Profit": [profit.round(0)], 
            "Profit per trade": [profit_per_trade.round(2)], 
            "Profit per trading day": [profit_per_day.round(2)], 
            "Sharpe ratio": [sharpe_ratio_monthly(df_year).round(2)], 
            "CAPM alpha": [capm_alpha(df_year).round(2)], 
            "Annualized return in percent": [annualized_return.round(2)], 
            "Max drawdown": [max_drawdown(df_year).round(2)]
        })

        # Add metrics to the results DataFrame
        results = pd.concat([results, year_metrics])

# Reset index of the results DataFrame
results.reset_index(drop=True, inplace=True)

# Print the results DataFrame
print(results)

2016-02-18 00:00:00
Number of long positions:  124
Number of short positions:  130
Balnace:  10048172.752677275
2021-07-08 00:00:00
Number of long positions:  1120
Number of short positions:  1120
Balnace:  9845303.259694645
   Year Starting balance  Ending balance    Profit  Profit per trade   
0  2015         10000000      10090252.0   90252.0             20.30  \
1  2016         10000000      10121725.0  121725.0             36.51   
2  2017         10000000      10266845.0  266845.0             53.27   
3  2018         10000000      10397126.0  397126.0             46.75   
4  2019         10000000      10283790.0  283790.0             50.60   
5  2020         10000000       9548759.0 -451241.0            -44.99   
6  2021         10000000      10233125.0  233125.0             23.69   
7  2022         10000000      10597327.0  597327.0             33.04   
8  2023         10000000       9839173.0 -160827.0            -99.83   

   Profit per trading day  Sharpe ratio  CAPM alpha   

In [25]:
display(results)

# Add a final row to the results DataFrame with the average metrics
results.loc["Average"] = results.mean()

# Turn year, starting balance and ending balance, profit into integers
results[["Year", "Starting balance", "Ending balance", "Profit"]] = results[["Year", "Starting balance", "Ending balance", "Profit"]].astype(int)
#Remove profit per trad and profit per trading day
results = results.drop(columns=["Profit per trade", "Profit per trading day"])
# Rename the columns to  Year &  Starting &  Ending &  Profit &  Sharpe  &  alpha &  Return &  MDD \\
results.columns = ["Year", "Starting", "Ending", "Profit", "Sharpe", "alpha", "Return", "MDD"]
# Round to 2 decimals
float_cols = ["Sharpe", "alpha", "Return", "MDD"]
# Turn the columns into floats
results[float_cols] = results[float_cols].astype(float)

# Rename the columns to  Year &  Starting &  Ending &  Profit &  Sharpe  &  alpha &  Return &  MDD \\
results.columns = ["Year", "Starting", "Ending", "Profit", "Sharpe", "alpha", "Return", "MDD"]

results[float_cols] = results[float_cols].round(2)
print(results.to_latex(index=False, float_format="%.2f"))

,Year,Starting balance,Ending balance,Profit,Profit per trade,Profit per trading day,Sharpe ratio,CAPM alpha,Annualized return in percent,Max drawdown
0,2015,10000000,10090252.0,90252.0,20.30,368.38,-0.38,0.01,0.93,-5.14
1,2016,10000000,10121725.0,121725.0,36.51,486.90,1.00,0.10,1.23,-18.32
2,2017,10000000,10266845.0,266845.0,53.27,1071.67,0.56,0.05,2.71,-2.25
3,2018,10000000,10397126.0,397126.0,46.75,1582.18,0.70,0.04,3.99,-5.10
4,2019,10000000,10283790.0,283790.0,50.60,1153.62,-0.06,0.02,2.85,-4.24
5,2020,10000000,9548759.0,-451241.0,-44.99,-1812.21,-1.23,-0.03,-4.52,-5.52
6,2021,10000000,10233125.0,233125.0,23.69,951.53,0.09,0.01,2.36,-3.44
7,2022,10000000,10597327.0,597327.0,33.04,2342.46,0.42,0.05,6.04,-9.30
8,2023,10000000,9839173.0,-160827.0,-99.83,-2725.88,-3.72,-0.08,-6.73,-1.85


,Year,Starting balance,Ending balance,Profit,Profit per trade,Profit per trading day,Sharpe ratio,CAPM alpha,Annualized return in percent,Max drawdown
0,2015,10000000,1.009025e+07,90252.000000,20.30,368.38,-0.380000,0.010000,0.930000,-5.140000
1,2016,10000000,1.012172e+07,121725.000000,36.51,486.90,1.000000,0.100000,1.230000,-18.320000
2,2017,10000000,1.026684e+07,266845.000000,53.27,1071.67,0.560000,0.050000,2.710000,-2.250000
3,2018,10000000,1.039713e+07,397126.000000,46.75,1582.18,0.700000,0.040000,3.990000,-5.100000
4,2019,10000000,1.028379e+07,283790.000000,50.60,1153.62,-0.060000,0.020000,2.850000,-4.240000
5,2020,10000000,9.548759e+06,-451241.000000,-44.99,-1812.21,-1.230000,-0.030000,-4.520000,-5.520000
6,2021,10000000,1.023312e+07,233125.000000,23.69,951.53,0.090000,0.010000,2.360000,-3.440000
7,2022,10000000,1.059733e+07,597327.000000,33.04,2342.46,0.420000,0.050000,6.040000,-9.300000
8,2023,10000000,9.839173e+06,-160827.000000,-99.83,-2725.88,-3.720000,-0.080000,-6.730000,-1.850000
Average,2019.0,10000000.0,1.015312e+07,153124.666667,13.26,379.85,-0.291111,0.018889,0.984444,-6.128889


In [33]:
# Turn year, starting balance and ending balance, profit into integers
results[["Year", "Starting balance", "Ending balance", "Profit"]] = results[["Year", "Starting balance", "Ending balance", "Profit"]].astype(int)
#Remove profit per trad and profit per trading day
results = results.drop(columns=["Profit per trade", "Profit per trading day"])
# Rename the columns to  Year &  Starting &  Ending &  Profit &  Sharpe  &  alpha &  Return &  MDD \\
results.columns = ["Year", "Starting", "Ending", "Profit", "Sharpe", "alpha", "Return", "MDD"]
# Round to 2 decimals
float_cols = ["Sharpe", "alpha", "Return", "MDD"]
# Turn the columns into floats
results[float_cols] = results[float_cols].astype(float)

# Rename the columns to  Year &  Starting &  Ending &  Profit &  Sharpe  &  alpha &  Return &  MDD \\
results.columns = ["Year", "Starting", "Ending", "Profit", "Sharpe", "alpha", "Return", "MDD"]

results[float_cols] = results[float_cols].round(2)
print(results.to_latex(index=False, float_format="%.2f"))

\begin{tabular}{rrrrrrrr}
\toprule
Year & Starting balance & Ending balance & Profit & Sharpe ratio & CAPM alpha & Annualized return in percent & Max drawdown \\
\midrule
2015 & 10000000 & 10090252 & 90252 & -0.38 & 0.01 & 0.93 & -5.14 \\
2016 & 10000000 & 10121725 & 121725 & 1.00 & 0.10 & 1.23 & -18.32 \\
2017 & 10000000 & 10266845 & 266845 & 0.56 & 0.05 & 2.71 & -2.25 \\
2018 & 10000000 & 10397126 & 397126 & 0.70 & 0.04 & 3.99 & -5.10 \\
2019 & 10000000 & 10283790 & 283790 & -0.06 & 0.02 & 2.85 & -4.24 \\
2020 & 10000000 & 9548759 & -451241 & -1.23 & -0.03 & -4.52 & -5.52 \\
2021 & 10000000 & 10233125 & 233125 & 0.09 & 0.01 & 2.36 & -3.44 \\
2022 & 10000000 & 10597327 & 597327 & 0.42 & 0.05 & 6.04 & -9.30 \\
2023 & 10000000 & 9839173 & -160827 & -3.72 & -0.08 & -6.73 & -1.85 \\
2019 & 10000000 & 10153124 & 153124 & -0.29 & 0.02 & 0.98 & -6.13 \\
\bottomrule
\end{tabular}



### Post analysis

In [ ]:
if run_optimizing == False:
    plot(df)

## Threshold search

In [ ]:
if run_optimizing:
    # Initialize wandb
    !pip install wandb
    import wandb
    wandb.login(key="b47bcf387a0571c5520c58a13be35cda8ada0a99")

    # Define the hyperparameters
    sweep_config = {
    'method': 'grid',
    'name': '2015-2018 - v12',
    'metric': {
        'goal': 'maximize', 
        'name': 'alpha'
        },
    'parameters': {
        'buy_threshold': {
            'values': [0.06, 0.1, 0.12]},
        'sell_threshold': {
            'values': [0.06, 0.1, 0.12]},
        'price_cap_lower': {
            'values': [1, 2]},
        'investment_ratio': {
            'values': [5e-4, 5e-5, 5e-6]},
        'long_short_ratio': {
            'values': [0.05, 0.3]}
        }
    }
    sweep_id = wandb.sweep(sweep=sweep_config, project='options-trading') 


In [ ]:
if run_optimizing:
    def find_best_thresholds(df = df):
        run = wandb.init(project = "options-trading")
        df = df.copy()
        buy_signal, sell_signal = generate_buy_sell_signals(df, run.config.buy_threshold, run.config.sell_threshold)
        df = trader(df, buy_signal, sell_signal, starting_balance=100000, price_cap_lower = run.config.price_cap_lower, investment_ratio = run.config.investment_ratio, long_short_ratio = run.config.long_short_ratio)
        df = calculate_options_value(df)
        run.log({"buy_threshold": run.config.buy_threshold, "sell_threshold": run.config.sell_threshold, "sharpe_ratio": sharpe_ratio_monthly(df), "ending_balance": df['Balance'].iloc[-1], "number_of_trades": df['Signal'].abs().sum(), "alpha": capm_alpha(df), "long_short_ratio": run.config.long_short_ratio, "price_cap_lower": run.config.price_cap_lower, "investment_ratio": run.config.investment_ratio})
        # Wandb callback
        print_results(df, 100000)
        plot(df)
        run.finish()

    wandb.agent(sweep_id=sweep_id, function=find_best_thresholds, project='options-trading', count = 1000)